# Introduction

swap out figure

<center>
<img src="https://i.imgur.com/yNp4qtV.png"><br/>
</center>

# Minimax

...

# Code

.

In [ ]:
#$HIDE_INPUT$
import random
import numpy as np

then they play against the agent

# Your turn

Continue to **[...link...](#$NEXT_NOTEBOOK_URL$)** ...